<a href="https://colab.research.google.com/github/kangwonlee/fdm_example/blob/main/fdm_si.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sample Python code simulating the behavior of a doped semiconductor using FDM:
* AskUp

This code simulates the behavior of a Si substrate with a doping profile of P impurities. It solves Poisson's equation and the continuity equation using FDM to calculate the electric potential and the electron and hole concentrations in the substrate. The simulation results are then plotted as a function of depth. Note that this is just a simple example, and more complex simulations may involve additional material properties and equations.

In [ ]:
import numpy as np

# Define simulation parameters
dx = 1e-9  # grid spacing in meters
dt = 1e-12  # time step in seconds
t_total = 1e-9  # total simulation time in seconds
V_bias = 1  # applied bias in volts

# Define material properties
eps0 = 8.854e-12  # vacuum permittivity
eps_r = 11.68  # relative permittivity of Si
q = 1.602e-19  # elementary charge
k_B = 1.38e-23  # Boltzmann constant

# Define doping profile
N_d = 1e18  # doping concentration in cm^-3
x_d = 200e-9  # depth of doping profile in meters
N = N_d * np.exp(-np.arange(0, x_d, dx) / x_d)  # doping profile in cm^-3

# Initialize simulation variables
V = np.zeros(N.shape)  # electric potential in volts
n = np.zeros(N.shape)  # electron concentration in cm^-3
p = np.zeros(N.shape)  # hole concentration in cm^-3

# Define boundary conditions
V[0] = V_bias
n[0] = N_d
p[0] = 0
n[-1] = 0
p[-1] = N_d

# Define FDM coefficients
a = eps0 * eps_r / dx**2
b = q / eps0 / eps_r
c = dx**2 / q / eps0 / eps_r / k_B

# Perform simulation
for t in np.arange(0, t_total, dt):
    # Calculate electric potential using Poisson's equation
    V[1:-1] = (a * (V[0:-2] + V[2:]) + b * (N[1:-1] - n[1:-1] + p[1:-1])) / (2 * a + b * dx**2)

    # Calculate electron and hole concentrations using continuity equation
    n[1:-1] = n[1:-1] + dt * (N[1:-1] * np.exp((V[1:-1] - V_bias) / (k_B * c)) - n[1:-1] / c) / dx
    p[1:-1] = p[1:-1] + dt * (N[1:-1] * np.exp(-(V[1:-1] - V_bias) / (k_B * c)) - p[1:-1] / c) / dx

# Plot simulation results
import matplotlib.pyplot as plt
plt.plot(np.arange(0, x_d, dx) * 1e9, n, label='n')
plt.plot(np.arange(0, x_d, dx) * 1e9, p, label='p')
plt.xlabel('Depth (nm)')
plt.ylabel('Concentration (cm^-3)')
plt.legend()
plt.show()
